In [4]:
import json
from websockets import connect

In [8]:
API_KEY = "305aacd3e8d0d03d07e8600ba4a041bf6cad19bfc0f083ea6ecff91b230486cd"


In [13]:
# Replace with your actual API key
# API_KEY = "YOUR_API_KEY"

async def get_candlestick_data():
    uri = "wss://coinswitch.co/api/ws"  # WebSocket endpoint URL
    async with connect(uri) as websocket:
        # Subscribe to the channel for the desired currency pair
        await websocket.send(json.dumps({"setOp": "subscribe", "args": ["spot-btcusdt"]}))

        while True:
            try:
                data = await websocket.recv()
                # Parse JSON data
                parsed_data = json.loads(data)

                if "k" in parsed_data:  # Check for candlestick data
                    # Extract relevant information (replace with specific keys you need)
                    open_price = parsed_data["k"]["o"]
                    high_price = parsed_data["k"]["h"]
                    low_price = parsed_data["k"]["l"]
                    close_price = parsed_data["k"]["c"]
                    volume = parsed_da["k"]["v"]

                    # Process the data (e.g., print, store in database)
                    print(f"Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}, Volume: {volume}")

            except Exception as e:
                print(f"Error: {e}")
                break  # Handle errors and disconnect if needed

if __name__ == "__main__":
    import asyncio
    asyncio.get_event_loop().run_until_complete(get_candlestick_data())


/Users/zeeshan/miniconda3/envs/tensorflow/lib/python3.10/genericpath.py:77: RuntimeWarning: coroutine 'get_candlestick_data' was never awaited
  m = tuple(map(os.fspath, m))


RuntimeError: This event loop is already running